# Mimikatz DC Sync
Detects Mimikatz DC sync security events

## Rule Content
```
- title: Mimikatz DC Sync
  id: 611eab06-a145-4dfa-a295-3ccc5c20f59a
  description: Detects Mimikatz DC sync security events
  status: experimental
  date: 2018/06/03
  modified: 2019/10/08
  author: Benjamin Delpy, Florian Roth
  references:
  - https://twitter.com/gentilkiwi/status/1003236624925413376
  - https://gist.github.com/gentilkiwi/dcc132457408cf11ad2061340dcb53c2
  tags:
  - attack.credential_access
  - attack.s0002
  - attack.t1003
  logsource:
    product: windows
    service: security
    category: null
  detection:
    selection:
      EventID: 4662
      Properties:
      - '*Replicating Directory Changes All*'
      - '*1131f6ad-9c07-11d1-f79f-00c04fc2dcd2*'
    filter1:
      SubjectDomainName: Window Manager
    filter2:
      SubjectUserName:
      - NT AUTHORITY*
      - '*$'
    condition: selection and not filter1 and not filter2
  falsepositives:
  - Valid DC Sync that is not covered by the filters; please report
  level: high

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-endpoint-winevent-security-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='(((event_id:"4662" AND object_properties.keyword:(*Replicating\ Directory\ Changes\ All* OR *1131f6ad\-9c07\-11d1\-f79f\-00c04fc2dcd2*)) AND (NOT (SubjectDomainName:"Window\ Manager"))) AND (NOT (SubjectUserName.keyword:(NT\ AUTHORITY* OR *$))))')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()